## Adder

In this tutorial, we will construct a 2-bit adder.

In [1]:
import magma as m
m.set_mantle_target("ice40")

In the last example, we created a Python function that created a full adder. 
In this example, we are going to use the built-in `FullAdder` from `Mantle`, our standard library of useful circuits.

In [2]:
from mantle import FullAdder

print('{} [{}]'.format(str(FullAdder), str(FullAdder.interface)))

import lattice ice40
import lattice mantle40
FullAdder ["I0", In(Bit), "I1", In(Bit), "CIN", In(Bit), "O", Out(Bit), "COUT", Out(Bit)]


Since we are using the ICE40, we need to set the target of `Mantle` to be the ICE40.

After we import `FullAdder` we print out its *interface*. This tells us that the full adder has three inputs `I0`, `I1`, and `CIN`. Note that the type of these arguments are `In(Bit)`. 
In `Magma` all arguments on an interface must be *qualified* to be inputs or outputs. The function `In` convert a `Magma` type to an input. There are also two outputs `O` and `COUT`. Note that the type of these are `Out(Bit)`.

Now let's build a 2-bit adder using `FullAdder`. 
We will built a simple ripple carry adder by connecting the carry out of one full adder
to the carry in of the next full adder.
The resulting adder will accept as input a carry in,
and generate a final carry out.

![2-Bit Adder](logisim/adder.png)

Here is a Python function that constructs the adder.

In [3]:
def adder(A, B, Cin):
    n = len(A)
    C = []
    Cout = Cin
    for i in range(n):
        fulladder = FullAdder()
        m.wire(A[i], fulladder.I0)
        m.wire(B[i], fulladder.I1)
        m.wire(Cout, fulladder.CIN)
        C.append(fulladder.O)
        Cout = fulladder.COUT
    return m.uint(C), Cout 

Note that we construct multiple instances of the full adder by calling `FullAdder()` inside the `for` loop. We also create a list of single bit outputs in the Python variable `C`. 
We then return this list converted to a `Magma` `Uint`. In addition to `Bits(n)`,
`Magma` also has built in types `UInt(n)` and `SInt(n)` to represent unsigned and signed ints.
`Magma` also has type conversion functions `bits`, `uint`, and `sint` to convert
between different types. 
In this example, `m.uint(C)` converts the list of bits to a `UInt(len(C))`.

In order to run test the adder,
we setup the IceStick board
to have four inputs and 2 outputs.
As before, `J1` will be used for inputs and `J3` for outputs.

In [4]:
N = 2

from loam.boards.icestick import IceStick

icestick = IceStick()
for i in range(N):
    icestick.J1[i].input().on()
    icestick.J1[i+N].input().on()
    
for i in range(N+1):
    icestick.J3[i].output().on()

In [5]:
main = icestick.DefineMain()

s, c = adder(main.J1[0:N], main.J1[N:N+N], 0)
m.wire( s, main.J3[:N] )
m.wire( c, main.J3[N] )

m.EndDefine()

In [6]:
m.compile('build/add', main, vendor="lattice")

compiling FullAdder
compiling main


In [7]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif add.blif' add.v
arachne-pnr -q -d 1k -o add.txt -p add.pcf add.blif 
icepack add.txt add.bin
iceprog add.bin

/Users/hanrahan/git/magmathon/notebooks/tutorial/build


init..
cdone: high
reset..
cdone: low
flash ID: 0x20 0xBA 0x16 0x10 0x00 0x00 0x23 0x51 0x73 0x10 0x23 0x00 0x15 0x00 0x26 0x06 0x06 0x15 0x9F 0x4E
file size: 32220
erase 64kB sector at 0x000000..
programming..
reading..
VERIFY OK
cdone: high
Bye.


You can test the program by connecting up some switches and LEDs to the headers. You should see the sum of the inputs displayed on the LEDs. First, we need to find out what pins `J1` and `J3` are wired up to. (Note: you can use `%` to execute shell commands inline in Jupyter notebooks)

In [8]:
%cat build/add.pcf

set_io J1[0] 112
set_io J1[1] 113
set_io J1[2] 114
set_io J1[3] 115
set_io J3[2] 60
set_io J3[1] 61
set_io J3[0] 62


In this example, we have J1 wire up to the four switch/LED circuits on the left, and J3 wired up to the three LED (no switch) circuits on the right
![](images/adder-seq/IMG_20180610_182758.jpg)
![](images/adder-seq/IMG_20180610_182802.jpg)
![](images/adder-seq/IMG_20180610_182806.jpg)
![](images/adder-seq/IMG_20180610_182811.jpg)
![](images/adder-seq/IMG_20180610_182819.jpg)
![](images/adder-seq/IMG_20180610_182823.jpg)
![](images/adder-seq/IMG_20180610_182827.jpg)